# Biostat 257 Homework 4

**Due May 28 @ 11:59PM**

**Elvis Cui** elviscuihan@g.ucla.edu

*PhD Student at Department of Biostatistics, UCLA*

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.6.0)
  CPU: Apple M1
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, westmere)


In this assignment, we continue with the linear mixed effects model (LMM) considered in HW2
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma}_i + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effects predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effects predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma}_i \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

The log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ is 
$$
    \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) = - \frac{n_i}{2} \log (2\pi) - \frac{1}{2} \log \det \boldsymbol{\Omega}_i - \frac{1}{2} (\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} (\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}),
$$
where
$$
    \boldsymbol{\Omega}_i = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \mathbf{L} \mathbf{L}^T \mathbf{Z}_i^T,
$$
Because the variance component parameter $\boldsymbol{\Sigma}$ has to be positive semidefinite, we use its Cholesky factor $\mathbf{L}$ as optimization variable. 

Given $m$ independent data points $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$, $i=1,\ldots,m$, we seek the maximum likelihood estimate (MLE) by maximizing the log-likelihood
$$
\ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2) = \sum_{i=1}^m \ell_i(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2).
$$
In this assignment, we use the nonlinear programming (NLP) approach for optimization. In HW5, we will derive a EM (expectation-maximization) algorithm for the same problem. 

In [2]:
# load necessary packages; make sure install them first
using BenchmarkTools, CSV, DataFrames, DelimitedFiles, Distributions
using Ipopt, LinearAlgebra, MathProgBase, MixedModels, NLopt
using Random, RCall, Revise

## Q1. (Optional, 30 bonus pts) Derivatives

NLP optimization solvers expect users to provide at least a function for evaluating objective value. If users can provide further information such as gradient and Hessian, the NLP solvers will be more stable and converge faster. Automatic differentiation tools are becoming more powerful but cannot apply to all problems yet.

1. Show that the gradient of $\ell_i$ is
\begin{eqnarray*}
\nabla_{\boldsymbol{\beta}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &=& \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i, \\
\nabla_{\sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &=& - \frac{1}{2} \operatorname{tr} (\boldsymbol{\Omega}_i^{-1}) + \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i, \\
\frac{\partial}{\partial \mathbf{L}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &=& - \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L},
\end{eqnarray*}
where $\mathbf{r}_i = \mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}$. 

2. Derive the observed information matrix and the expected (Fisher) information matrix.

If you need a refresher on multivariate calculus, my [Biostat 216 lecture notes](https://ucla-biostat216-2019fall.github.io/slides/16-matrixcalc/16-matrixcalc.html) may be helpful.

## Sol Q1

### Sol Q1.1

**Step 1**: I need to cite several results from [Biostat 279](https://github.com/ElvisCuiHan/optimal-design-theory.git) and [Biostat 216 lecture notes](https://ucla-biostat216-2019fall.github.io/slides/16-matrixcalc/16-matrixcalc.html).

- (216) $D \phi(\mathbf{x})=\nabla \phi(\mathbf{x})^T$; $D \phi(\mathbf{X})=\left(\text{ vec }\frac{\partial\phi(\mathbf{x})}{\partial\mathbf{X}}\right)^T$ and the chain rule
$$D_{\mathbf{X}}\mathbf{Z}=D_\mathbf{Y}\mathbf{Z}\cdot D_\mathbf{X}\mathbf{Y}$$
- (216) For a scalr-valued matrix function $\phi(\mathbf{X})$, we have
$$D𝜙(𝐗)=(vec\frac{∂𝜙(𝐗)}{∂𝐗})′$$
- (279) Let $A(\alpha)$ be a matrix and its entries are functions of a scalar $\alpha$, then:
\begin{align}
\frac{d}{d\alpha}A^{-1}(\alpha)&=-A^{-1}(\alpha)\frac{dA(\alpha)}{d\alpha}A^{-1}(\alpha)\\
\frac{d}{d\alpha}\log\det A(\alpha)&=\text{Tr}\left(A^{-1}(\alpha)\frac{dA(\alpha)}{d\alpha}\right)
\end{align}
- (216) If $\phi(\mathbf{X})=\text{Tr}(\mathbf{AX}^{-1})$, then
$$\frac{\partial}{\partial\mathbf{X}}\phi(\mathbf{X})=-(\mathbf{X}^{-1}\mathbf{A}\mathbf{X}^{-1})^T$$

- (216) If $\phi(\mathbf{x})=\mathbf{x}^T\mathbf{Ax}$, then
$$\frac{\partial}{\partial\mathbf{X}}\phi(\mathbf{X})=\mathbf{x}^T(\mathbf{A}+\mathbf{A}^T)$$

**Step 2** $\nabla_\beta$ and $\nabla_{\sigma^2}$: Let 
$$f(\sigma^2,\mathbf{L},\beta)=(\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} (\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})=\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{r}_i$$ and $$g(\sigma^2,\mathbf{L})=\log\det(\mathbf{\Omega}_i)$$ then by the chain rule and other results from 216 and 279:
$$\nabla_\beta f=-2\mathbf{X}_i^T\mathbf{\Omega}_i^{-1}\mathbf{r}_i$$
For $\sigma^2$, re-write $\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1}\mathbf{r}_i=\text{Tr}\left(\mathbf{r}_i\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1}\right)$. Then I need to apply the first result:
\begin{align}
\nabla_{\sigma^2} f&=-\left(\text{vec }(\boldsymbol{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1})\right)^T(\text{vec }\mathbf{I}_{n_i})\\
&=-\text{Tr}\left[\boldsymbol{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-1})\right]\\
&=-\mathbf{r}_i^T\boldsymbol{\Omega}_i^{-2}\mathbf{r}_i
\end{align}
Similarly, by 279 result,
\begin{align}
\nabla_{\sigma^2} g&=\text{Tr}(\boldsymbol{\Omega}_i^{-1}\mathbf{I}_{n_i})=\text{Tr}(\boldsymbol{\Omega}_i^{-1})
\end{align}
In conclusion,
\begin{align}
\nabla_{\sigma^2} \ell_i&=-\frac{1}{2}\nabla_{\sigma^2} g-\frac{1}{2}\nabla_{\sigma^2} f\\
&=- \frac{1}{2} \operatorname{Tr} (\boldsymbol{\Omega}_i^{-1}) + \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i\\
\nabla_{\beta} \ell_i&=-\frac{1}{2}\nabla_{\beta} f\\
&=\mathbf{X}_i^T\mathbf{\Omega}_i^{-1}\mathbf{r}_i
\end{align}

### Step 3 
**Part (a)** $\nabla_L \log\det\mathbf{\Omega}^{-1}_i$: For this one, I need an additional result from 216: 

If $𝐹(𝐗)=𝐀𝐺(𝐗)𝐁𝐻(𝐗)𝐂$, then
$$D𝐹(𝐗)=(𝐂′𝐻(𝐗)′𝐁′⊗𝐀)D𝐺(𝐗)+(𝐂′⊗𝐀𝐺(𝐗)𝐁)D𝐻(𝐗).$$

and another result from [this website](https://ccrma.stanford.edu/~dattorro/matrixcalc.pdf):
$$∇_X a^T\mathbf{XX}^Tb = (ab^T + ba^T)\mathbf{X}$$

Plug-in $\mathbf{X}=\mathbf{L},\mathbf{A}=\mathbf{Z}_i, \mathbf{B}=\mathbf{I}_{p}$ and $\mathbf{C}=\mathbf{Z}_i^T$ and let 
$$G(\mathbf{\mathbf{L}})=L,\ H(\mathbf{\mathbf{L}})=L^T$$

Then I have
$$\frac{\partial \mathbf{\Omega}_i}{\partial \mathbf{L}}=\mathbf{Z}_i\mathbf{L}\otimes \mathbf{Z}_i+\left(\mathbf{Z}_i\otimes\mathbf{Z}_i\mathbf{L}\right)\mathbf{K}$$

where $\mathbf{K}$ is the $qq\times qq$ comutation matrix.

Next, by **Roth column lemma**,

$$\left(\text{vec}\frac{\partial \log\det\mathbf{\Omega}_i^{-1}}{\partial\mathbf{\Omega}_i}\right)^T\frac{\partial \mathbf{\Omega}_i}{\partial \mathbf{L}}=2(\text{vec }\mathbf{L}^T\mathbf{Z}_i^T\mathbf{\Omega}_i^{-1}\mathbf{Z}_i)^T$$

Re-shape it into matrix form:
$$\frac{\partial g(\sigma^2,\mathbf{L})}{\partial\mathbf{L}}=2\mathbf{Z}_i^T\mathbf{\Omega}_i^{-1}\mathbf{Z}_i\mathbf{L}$$

**Part (b)** $\nabla_Lr_i^T\mathbf{\Omega}_i^{-1}r_i$: For this one, all I need are the ***chain rule*** and the following formula.

- (216) If $\phi(\mathbf{X})=\text{Tr}(\mathbf{AX}^{-1})$, then
$$\frac{\partial}{\partial \mathbf{X}}\phi(\mathbf{X})=-(\mathbf{X}^{-1}\mathbf{A}\mathbf{X}^{-1})^T$$

Note that $$\mathbf{r}_i^T\mathbf{\Omega}_i^{-1}\mathbf{r}_i=\text{Tr}(\mathbf{r}_i\mathbf{r}_i^T\mathbf{\Omega}_i^{-1})$$ 
Thus,

$$\frac{\partial}{\partial \mathbf{\Omega}_i}\mathbf{r}_i^T\mathbf{\Omega}_i^{-1}\mathbf{r}_i=-\mathbf{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T\mathbf{\Omega}_i^{-1}$$

Hence, again by **Roth column lemma**
$$\left(\text{vec }\frac{\partial}{\partial \mathbf{\Omega}_i}\mathbf{r}_i^T\mathbf{\Omega}_i^{-1}\mathbf{r}_i\right)^T\frac{\partial \mathbf{\Omega}_i}{\partial \mathbf{L}}=-2(\text{vec }\mathbf{L}^T\mathbf{Z}_i^T\mathbf{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T\mathbf{\Omega}_i^{-1}\mathbf{Z}_i)^T$$

Put it into matrix form:
$$\frac{\partial \mathbf{r}_i^T\mathbf{\Omega}_i^{-1}\mathbf{r}_i}{\partial\mathbf{L}}=-2\mathbf{Z}_i^T\mathbf{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T\mathbf{\Omega}_i^{-1}\mathbf{Z}_i\mathbf{L}$$

**Part (3)** $\nabla_L \ell_i$: Combine part (a) and part (b), I get
\begin{align}
\nabla_\mathbf{L} \ell_i&=-\mathbf{Z}_i^T\mathbf{\Omega}_i^{-1}\mathbf{Z}_i\mathbf{L}+\mathbf{Z}_i^T\mathbf{\Omega}_i^{-1}\mathbf{r}_i\mathbf{r}_i^T\mathbf{\Omega}_i^{-1}\mathbf{Z}_i\mathbf{L}
\end{align}

### Sol Q1.2 (unfinished)

Recall that

\begin{align}
\nabla_{\beta} \ell_i&=\mathbf{X}_i^T\mathbf{\Omega}_i^{-1}\mathbf{r}_i\\
\nabla_{\sigma^2} \ell_i
&=- \frac{1}{2} \operatorname{Tr} (\boldsymbol{\Omega}_i^{-1}) + \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i\\
\nabla_{\mathbf{L}} \ell_i&=- \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L}
\end{align}

#### Observed Information Matrix 
\begin{align}
\mathcal{M}(\beta,\sigma^2,\mathbf{L})&=\sum_{i=1}^n\left[\begin{matrix}
\nabla_{\beta} \ell_i \nabla^T_{\beta} \ell_i & \nabla_{\beta} \ell_i\nabla_{\sigma^2} \ell_i &\nabla_{\beta} \ell_i\nabla^T_{\mathbf{L}} \ell_i\\
\nabla_{\sigma^2} \ell_i\nabla_{\beta} \ell_i&\nabla_{\sigma^2} \ell_i\nabla_{\sigma^2} \ell_i&\nabla_{\sigma^2} \ell_i\nabla^T_{\mathbf{L}} \ell_i\\
\nabla_{\mathbf{L}} \ell_i\nabla^T_{\beta} \ell_i&\nabla_{\mathbf{L}} \ell_i\nabla_{\sigma^2} \ell_i&\nabla_{\mathbf{L}} \ell_i\nabla^T_{\mathbf{L}} \ell_i
\end{matrix}\right]
\end{align}
where now $\nabla_{\mathbf{L}} \ell_i$ is the **vectorized** version.

#### Fisher Information Matrix
$$\mathbb{E}\mathcal{M}(\beta,\sigma^2,\mathbf{L})$$
where the expectation is taken w.r.t. $\mathbf{y}_i,i=1,2,\cdots,n$.

## Q2. (20 pts) Objective and gradient evaluator for a single datum

We expand the code from HW2 to evaluate both objective and gradient. I provide my code for HW2 below as a starting point. You do _not_ have to use this code. If your come up faster code, that's even better. 

### Sol Q2: Supplementary materials

- $\nabla_{\beta} \ell_i$: By SWM formula in HW2, re-write (drop all unnecessary sub-indexes):
\begin{align}
\mathbf{X}^T\mathbf{\Omega}^{-1}\mathbf{r}&=\frac{1}{\sigma^2}\left((X^Ty-X^TX\beta)-\underbrace{X^TZL}_{(1)}\underbrace{U^{-1}U^{-T}L^TZ^Tr}_{(2)}\right)
\end{align}

where
\begin{align}
U&=\text{chol}(σ²I + L^TZ^TZL)\\
(1)&=\text{transpose(ztx)}L\\
(2)&=U\ \text{\\ storage_q}
\end{align}

- $\nabla_{\sigma^2} \ell_i$: The key is to compute the 2 quantities.

Again by SWM formula,
\begin{align}
\text{Tr}(\Omega^{-1})&=\frac{n}{\sigma^2}-\frac{1}{\sigma^2}\text{Tr}\left(\underbrace{L^TZ^T}_{A^T}ZL(\sigma^2 I_q+L^TZ^TZL)^{-1}\right)\\
&=\frac{n}{\sigma^2}-\frac{1}{\sigma^2}\text{Tr}\left(I_q+\sigma^2(A^TA)^{-1}\right)^{-1}
\end{align}
Next, by SWM formula again,
\begin{align}
\left(I_q+\sigma^2(A^TA)^{-1}\right)^{-1}&=I_q-\left(\frac{1}{\sigma^2} (A^TA) + I_q\right)^{-1}
\end{align}
I get

$$\text{Tr}(\Omega^{-1})=\frac{n-q}{\sigma^2}-\text{Tr}(\sigma^2I_q+A^TA)^{-1}$$

In [110]:
# define a type that holds an LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y          :: Vector{T}
    X          :: Matrix{T}
    Z          :: Matrix{T}
    # arrays for holding gradient
    ∇β         :: Vector{T}
    ∇σ²        :: Vector{T}
    ∇Σ         :: Matrix{T}    
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate
    yty        :: T
    xty        :: Vector{T}
    zty        :: Vector{T}
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    storage_q2 :: Vector{T}
    storage_q3 :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    storage_qq :: Matrix{T}
    storage_qq2:: Matrix{T}
    storage_Iq :: Matrix{T} 
end

"""
    LmmObs(y::Vector, X::Matrix, Z::Matrix)

Create an LMM datum of type `LmmObs`.
"""
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}
    ) where T <: AbstractFloat
    n, p, q    = size(X, 1), size(X, 2), size(Z, 2)    
    ∇β         = Vector{T}(undef, p)
    ∇σ²        = Vector{T}(undef, 1)
    ∇Σ         = Matrix{T}(undef, q, q)    
    yty        = abs2(norm(y))
    xty        = transpose(X) * y
    zty        = transpose(Z) * y    
    storage_p  = Vector{T}(undef, p)
    storage_q  = Vector{T}(undef, q)
    storage_q2 = Vector{T}(undef, q)
    storage_q3 = Vector{T}(undef, q)
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    storage_qq = similar(ztz)
    storage_qq2= similar(ztz)
    storage_Iq = 1.0 * Matrix{T}(I, q, q) 
    LmmObs(y, X, Z, ∇β, ∇σ², ∇Σ, 
        yty, xty, zty, storage_p, storage_q, storage_q2, storage_q3,
        xtx, ztx, ztz, storage_qq, storage_qq2, storage_Iq)
end

"""
    logl!(obs::LmmObs, β, L, σ², needgrad=false)

Evaluate the log-likelihood of a single LMM datum at parameter values `β`, `L`, 
and `σ²`. If `needgrad==true`, then `obs.∇β`, `obs.∇Σ`, and `obs.σ² are filled 
with the corresponding gradient.
"""
function logl!(
        obs      :: LmmObs{T}, 
        β        :: Vector{T}, 
        L        :: Matrix{T}, 
        σ²       :: T,
        needgrad :: Bool = true
    ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    ####################
    # Evaluate objective
    ####################    
    # form the q-by-q matrix: M = σ² * I + Lt Zt Z L
    copy!(obs.storage_qq, obs.ztz)
    BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.storage_qq) # O(q^3)
    BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.storage_qq) # O(q^3)
    @inbounds for j in 1:q
        obs.storage_qq[j, j] += σ²
    end
    # cholesky on M = σ² * I + Lt Zt Z L
    LAPACK.potrf!('U', obs.storage_qq) # O(q^3)
    # storage_q = (Mchol.U') \ (Lt * (Zt * res))
    BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), copy!(obs.storage_q, obs.zty)) # O(pq)
    BLAS.trmv!('L', 'T', 'N', L, obs.storage_q)    # O(q^2)
    BLAS.trsv!('U', 'T', 'N', obs.storage_qq, obs.storage_q) # O(q^3)
    # l2 norm of residual vector
    copy!(obs.storage_p, obs.xty)
    rtr  = obs.yty +
        dot(β, BLAS.gemv!('N', T(1), obs.xtx, β, T(-2), obs.storage_p))
    # assemble pieces
    logl::T = n * log(2π) + (n - q) * log(σ²) # constant term
    @inbounds for j in 1:q
        logl += 2log(obs.storage_qq[j, j])
    end
    qf    = abs2(norm(obs.storage_q)) # quadratic form term
    logl += (rtr - qf) / σ² 
    logl /= -2
    ###################
    # Evaluate gradient
    ###################    
    if needgrad
        # TODO: fill ∇β, ∇L, ∇σ² by gradients
        # sleep(1e-3) # pretend this step takes 1ms
        
        # STEP 1:
        # ∇β = X inv(Ω) r
        ## X^TZLU^{-1}storage_q / σ²
        copy!(obs.storage_q2, obs.storage_q)
        BLAS.trsv!('U', 'N', 'N', obs.storage_qq, obs.storage_q2)
        copy!(obs.storage_q3, obs.storage_q2)
        BLAS.trmv!('L', 'N', 'N', L, obs.storage_q2)
        BLAS.gemv!('T', -1/σ², obs.ztx, obs.storage_q2, 0.0, obs.∇β)
        ## (X^Ty-X^TXβ) / σ²
        BLAS.gemv!('N', -1/σ², obs.xtx, β, 1/σ², copy!(obs.storage_p, obs.xty))
        ## Put them together
        axpy!(1.0, obs.storage_p, obs.∇β)
        
        # STEP 2:
        # ∇σ² = -1/2 * Tr(inv(Ω)) + 1/2 * r^T (inv(Ω))^2 r
        ## (q-n)/σ² + Tr(inv(σ²I+A^TA))/σ²
        copy!(obs.storage_qq2, obs.storage_Iq)
        BLAS.trsm!('L', 'U', 'T', 'N', 1.0, obs.storage_qq, obs.storage_qq2)
        obs.∇σ²[1] = (-n + q) / σ² - sum(abs2, obs.storage_qq2)
        ## r^T (inv(Ω))^2 r
        obs.∇σ²[1] += obs.yty / (σ²)^2
        copy!(obs.storage_p, obs.xty)
        obs.∇σ²[1] += dot(β, BLAS.gemv!('N', T(1), obs.xtx, β, T(-2), obs.storage_p)) / (σ²)^2
        obs.∇σ²[1] -= sum(abs2, obs.storage_q3)/σ²
        obs.∇σ²[1] -= sum(abs2, obs.storage_q)/(σ²)^2
        obs.∇σ²[1] /= (2.0)
        
        # STEP 3:
        # ∇L = −𝐙^Tinv(𝛀)𝐙+𝐙^Tinv(𝛀)𝐫𝐫^Tinv(𝛀)𝐙
        ## −𝐙^Tinv(𝛀)𝐙
        copy!(obs.storage_qq2, obs.ztz)
        BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.storage_qq2)
        BLAS.trsm!('L', 'U', 'T', 'N', 1.0, obs.storage_qq, obs.storage_qq2)
        mul!(obs.∇Σ, transpose(obs.storage_qq2), obs.storage_qq2)
        #rmul!(obs.∇Σ, LowerTriangular(L))
        mul!(obs.∇Σ, obs.ztz, I, T(1), T(-1))
        rdiv!(obs.∇Σ, -σ²)
        ## 𝐙^Tinv(𝛀)𝐫𝐫^Tinv(𝛀)𝐙
        ### 𝐙^Tinv(𝛀)𝐫
        mul!(obs.storage_q, obs.ztz, obs.storage_q2)
        mul!(obs.storage_q, obs.ztx, β, -1.0, -1.0)
        axpy!(1.0, obs.zty, obs.storage_q)
        rdiv!(obs.storage_q, σ²)
        ### Rank 1 update
        BLAS.ger!(1.0, obs.storage_q, obs.storage_q, obs.∇Σ)
    end    
    ###################
    # Return
    ###################        
    return logl    
end

logl!

It is a good idea to test correctness and efficiency of the single datum objective/gradient evaluator here. First generate the same data set as in HW2.

In [111]:
Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I # compound symmetry 
L  = Matrix(cholesky(Symmetric(Σ)).L)
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form the LmmObs object
obs = LmmObs(y, X, Z);

### Correctness

In [112]:
@show logl = logl!(obs, β, L, σ², true)
@show obs.∇β
@show obs.∇Σ
@show obs.∇σ²;

logl = logl!(obs, β, L, σ², true) = -3261.917755918759
obs.∇β = [-1.1937579038483364, -20.068104130062565, -12.32394232952253, -5.174641191561896, 28.20847569099034]
obs.∇Σ = [0.40732856643463355 -0.103704571942902 0.9214023976868062; -0.103704571942902 -0.9907411137749693 -0.021658311185031615; 0.9214023976868062 -0.021658311185031615 -0.5355581080214666]
obs.∇σ² = [5.501620540891281]


You will lose all 20 points if following statement throws `AssertionError`.

In [113]:
@assert abs(logl - (-3261.9177559187597)) < 1e-8
@assert norm(obs.∇β - [-1.1937579038479573, -20.068104130062466, 
        -12.323942329522424, -5.174641191562253, 28.208475690990248]) < 1e-8
@assert norm(obs.∇Σ - [0.40732856643442306 -0.10370457194285436 0.9214023976868783; 
        -0.10370457194285436 -0.9907411137746756 -0.02165831118509598; 
        0.9214023976868783 -0.02165831118509598 -0.5355581080215592]) < 1e-8
@assert abs(obs.∇σ²[1] - (5.501620540891395)) < 1e-8

### Efficiency

Benchmark for evaluating objective only. This is what we did in HW2.

In [7]:
@benchmark logl!($obs, $β, $L, $σ², false)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     649.646 ns (0.00% GC)
  median time:      664.378 ns (0.00% GC)
  mean time:        666.580 ns (0.00% GC)
  maximum time:     1.344 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     164

Benchmark for objective + gradient evaluation.

In [8]:
bm_objgrad = @benchmark logl!($obs, $β, $L, $σ², true)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.533 μs (0.00% GC)
  median time:      1.575 μs (0.00% GC)
  mean time:        1.597 μs (0.00% GC)
  maximum time:     9.246 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     10

My median runt time is 2.1μs. You will get full credit (10 pts) if the median run time is within 10μs.

In [9]:
#  The points you will get are
clamp(10 / (median(bm_objgrad).time / 1e3) * 10, 0, 10)

10.0

In [10]:
# # check for type stability
# @code_warntype logl!(obs, β, L, σ², true)

In [11]:
# using Profile

# Profile.clear()
# @profile for i in 1:10000; logl!(obs, β, L, σ², true); end
# Profile.print(format=:flat)

## Q3. LmmModel type

We create a `LmmModel` type to hold all data points and model parameters. Log-likelihood/gradient of a `LmmModel` object is simply the sum of log-likelihood/gradient of individual data points. 

In [12]:
# define a type that holds LMM model (data + parameters)
struct LmmModel{T <: AbstractFloat} <: MathProgBase.AbstractNLPEvaluator
    # data
    data :: Vector{LmmObs{T}}
    # parameters
    β    :: Vector{T}
    L    :: Matrix{T}
    σ²   :: Vector{T}    
    # arrays for holding gradient
    ∇β   :: Vector{T}
    ∇σ²  :: Vector{T}
    ∇L   :: Matrix{T}
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    xty  :: Vector{T}
    ztr  :: Vector{T}
    ztr2 :: Vector{T}
    xtx  :: Matrix{T}
    ztz  :: Matrix{T}
    ztz2 :: Matrix{T}
    Σ    :: Matrix{T}
end

"""
    LmmModel(data::Vector{LmmObs})

Create an LMM model that contains data and parameters.
"""
function LmmModel(obsvec::Vector{LmmObs{T}}) where T <: AbstractFloat
    # dims
    p    = size(obsvec[1].X, 2)
    q    = size(obsvec[1].Z, 2)
    # parameters
    β    = Vector{T}(undef, p)
    L    = Matrix{T}(undef, q, q)
    σ²   = Vector{T}(undef, 1)    
    # gradients
    ∇β   = similar(β)    
    ∇σ²  = similar(σ²)
    ∇L   = similar(L)
    # intermediate arrays
    xty  = Vector{T}(undef, p)
    ztr  = Vector{T}(undef, q)
    ztr2 = Vector{T}(undef, abs2(q))
    xtx  = Matrix{T}(undef, p, p)
    ztz  = Matrix{T}(undef, q, q)
    ztz2 = Matrix{T}(undef, abs2(q), abs2(q))
    Σ    = Matrix{T}(undef, q, q)
    LmmModel(obsvec, β, L, σ², ∇β, ∇σ², ∇L, xty, ztr, ztr2, xtx, ztz, ztz2, Σ)
end

"""
    logl!(m::LmmModel, needgrad=false)

Evaluate the log-likelihood of an LMM model at parameter values `m.β`, `m.L`, 
and `m.σ²`. If `needgrad==true`, then `m.∇β`, `m.∇Σ`, and `m.σ² are filled 
with the corresponding gradient.
"""
function logl!(m::LmmModel{T}, needgrad::Bool = false) where T <: AbstractFloat
    logl = zero(T)
    if needgrad
        fill!(m.∇β , 0)
        fill!(m.∇L , 0)
        fill!(m.∇σ², 0)        
    end
    @inbounds for i in 1:length(m.data)
        obs = m.data[i]
        logl += logl!(obs, m.β, m.L, m.σ²[1], needgrad)
        if needgrad
            BLAS.axpy!(T(1), obs.∇β, m.∇β)
            BLAS.axpy!(T(1), obs.∇Σ, m.∇L)
            m.∇σ²[1] += obs.∇σ²[1]
        end
    end
    # obtain gradient wrt L: m.∇L = m.∇L * L
    if needgrad
       # TODO 
        BLAS.trmm!('R', 'L', 'N', 'N', T(1), m.L, m.∇L)
    end
    logl
end

logl!

## Q4. (20 pts) Test data

Let's generate a fake longitudinal data set to test our algorithm.

In [13]:
Random.seed!(257)

# dimension
m      = 1000 # number of individuals
ns     = rand(1500:2000, m) # numbers of observations per individual
p      = 5 # number of fixed effects, including intercept
q      = 3 # number of random effects, including intercept
obsvec = Vector{LmmObs{Float64}}(undef, m)
# true parameter values
βtrue  = [0.1; 6.5; -3.5; 1.0; 5]
σ²true = 1.5
σtrue  = sqrt(σ²true)
Σtrue  = Matrix(Diagonal([2.0; 1.2; 1.0]))
Ltrue  = Matrix(cholesky(Symmetric(Σtrue)).L)
# generate data
for i in 1:m
    # first column intercept, remaining entries iid std normal
    X = Matrix{Float64}(undef, ns[i], p)
    X[:, 1] .= 1
    @views Distributions.rand!(Normal(), X[:, 2:p])
    # first column intercept, remaining entries iid std normal
    Z = Matrix{Float64}(undef, ns[i], q)
    Z[:, 1] .= 1
    @views Distributions.rand!(Normal(), Z[:, 2:q])
    # generate y
    y = X * βtrue .+ Z * (Ltrue * randn(q)) .+ σtrue * randn(ns[i])
    # form a LmmObs instance
    obsvec[i] = LmmObs(y, X, Z)
end
# form a LmmModel instance
lmm = LmmModel(obsvec);

For later comparison with other software, we save the data into a text file `lmm_data.csv`. **Do not put this file in Git.** It takes 246.6MB storage.

In [14]:
(isfile("lmm_data.csv") && filesize("lmm_data.csv") == 244697920) || 
open("lmm_data.csv", "w") do io
    p = size(lmm.data[1].X, 2)
    q = size(lmm.data[1].Z, 2)
    # print header
    print(io, "ID,Y,")
    for j in 1:(p-1)
        print(io, "X" * string(j) * ",")
    end
    for j in 1:(q-1)
        print(io, "Z" * string(j) * (j < q-1 ? "," : "\n"))
    end
    # print data
    for i in eachindex(lmm.data)
        obs = lmm.data[i]
        for j in 1:length(obs.y)
            # id
            print(io, i, ",")
            # Y
            print(io, obs.y[j], ",")
            # X data
            for k in 2:p
                print(io, obs.X[j, k], ",")
            end
            # Z data
            for k in 2:q-1
                print(io, obs.Z[j, k], ",")
            end
            print(io, obs.Z[j, q], "\n")
        end
    end
end

### Correctness

Evaluate log-likelihood and gradient of whole data set at the true parameter values.

In [15]:
copy!(lmm.β, βtrue)
copy!(lmm.L, Ltrue)
lmm.σ²[1] = σ²true
@show obj = logl!(lmm, true)
@show lmm.∇β
@show lmm.∇σ²
@show lmm.∇L;

obj = logl!(lmm, true) = -2.834275229633783e6
lmm.∇β = [11.772120067027387, -1925.6905435286428, -590.4811426626873, 2112.4417700625054, 2261.4604838394603]
lmm.∇σ² = [911.8747620566039]
lmm.∇L = [24.419901311348944 24.011922772751674 16.56376848155736; 30.99925900309529 -59.40190266155378 46.583314134721995; 23.424706030626428 51.029463909872966 12.038870789805124]


Test correctness. You will loss all 20 points if following code throws `AssertError`.

In [16]:
@assert abs(obj - (-2.8342752296337797e6)) < 1e-6
@assert norm(lmm.∇β - [11.772120067073923, -1925.690543528459, 
        -590.4811426628461, 2112.441770062488, 2261.4604838395016]) < 1e-6
@assert norm(lmm.∇L - [24.419901311377576 24.01192277273847 16.563768481574012; 
        30.99925900307824 -59.40190266157969 46.58331413470851; 
        23.424706030649975 51.029463909858194 12.038870789777398]) < 1e-6
@assert abs(lmm.∇σ²[1] - (911.8747620554551)) < 1e-6

### Efficiency

Test efficiency.

In [17]:
bm_model = @benchmark logl!($lmm, true)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.611 ms (0.00% GC)
  median time:      1.619 ms (0.00% GC)
  mean time:        1.622 ms (0.00% GC)
  maximum time:     1.768 ms (0.00% GC)
  --------------
  samples:          3080
  evals/sample:     1

My median run time is 2.122ms. You will get full credit if your median run time is within 10ms. The points you will get are

In [18]:
clamp(10 / (median(bm_model).time / 1e6) * 10, 0, 10)

10.0

### Memory

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [19]:
clamp(10 - median(bm_model).memory / 100, 0, 10)

10.0

## Q5. (30 pts) Starting point

For numerical optimization, a good starting point is critical. Let's start $\boldsymbol{\beta}$ and $\sigma^2$ from the least sqaures solutions (ignoring intra-individual correlations)
\begin{eqnarray*}
\boldsymbol{\beta}^{(0)} &=& \left(\sum_i \mathbf{X}_i^T \mathbf{X}_i\right)^{-1} \left(\sum_i \mathbf{X}_i^T \mathbf{y}_i\right) \\
\sigma^{2(0)} &=& \frac{\sum_i \|\mathbf{r}_i^{(0)}\|_2^2}{\sum_i n_i} = \frac{\sum_i \|\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}^{(0)}\|_2^2}{\sum_i n_i}.
\end{eqnarray*}
To get a reasonable starting point for $\boldsymbol{\Sigma} = \mathbf{L} \mathbf{L}^T$, we can minimize the least squares criterion (ignoring the noise variance component)
$$
    \text{minimize} \sum_i \| \mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T \|_{\text{F}}^2.
$$
Derive the minimizer $\boldsymbol{\Sigma}^{(0)}$ (10 pts). 

We implement this start point strategy in the function `init_ls()`.

## Sol Q5

Look @ a single term first:
\begin{align}
\lVert \mathbf{r}\mathbf{r}^T-\mathbf{Z\Sigma Z}^T \lVert^2_F&=\text{Tr}\left[(\mathbf{r}\mathbf{r}^T-\mathbf{Z\Sigma Z}^T)^T(\mathbf{r}\mathbf{r}^T-\mathbf{Z\Sigma Z}^T)\right]\\
&=\underbrace{\text{Tr}(\mathbf{rr}^T\mathbf{rr}^T)}_{(1)} - \underbrace{\text{Tr}(2\mathbf{Z}^T\mathbf{rr}^T\mathbf{Z\Sigma })}_{(2)}+\underbrace{\text{Tr}(\mathbf{Z\Sigma Z}^T\mathbf{Z\Sigma Z}^T)}_{(3)}
\end{align}
Then by [Dr. Zhou's Biostat 216 notes](https://ucla-biostat216-2019fall.github.io/slides/16-matrixcalc/16-matrixcalc.html):

\begin{align}
\frac{\partial (1)}{\partial\mathbf{\Sigma}}&=\mathbf{O}\\
\frac{\partial (2)}{\partial\mathbf{\Sigma}}&=2\mathbf{Z}^T\mathbf{rr}^T\mathbf{Z} \\
\frac{\partial (3)}{\partial\mathbf{\Sigma}}&=2\mathbf{Z}^T\mathbf{Z\Sigma Z}^T\mathbf{Z}
\end{align}

Set $\frac{\partial (1)}{\partial\mathbf{\Sigma}}-\frac{\partial (2)}{\partial\mathbf{\Sigma}}+\frac{\partial (3)}{\partial\mathbf{\Sigma}}=0$, and then sum over $i=1,2,\cdots,n$:

$$\sum_{i=1}^n\mathbf{Z_i}^T\mathbf{r}_i\mathbf{r}_i^T\mathbf{Z}_i=\sum_{i=1}^n\mathbf{Z}_i^T\mathbf{Z}_i\mathbf{\Sigma}\mathbf{Z}_i^T\mathbf{Z}_i$$

Next, by **Roth column lemma** in [Dr. Zhou's Notes on Easy Linear Systems](https://ucla-biostat-257-2021spring.github.io/slides/19-easylineq/easylineq.html),
$$\text{vec} \mathbf{ABC}=(\mathbf{C}^T\otimes\mathbf{A})\text{vec}\mathbf{B}$$
Thus,

$$\text{vec}\left(\sum_{i=1}^n\mathbf{Z}_i^T\mathbf{Z}_i\mathbf{\Sigma}\mathbf{Z}_i^T\mathbf{Z}_i\right)
=\sum_{i=1}^n\left(\mathbf{Z}_i^T\mathbf{Z}_i\otimes\mathbf{Z}_i^T\mathbf{Z}_i\right)\text{vec}\mathbf{\Sigma}=\sum_{i=1}^n\left(\mathbf{Z}_i^T\mathbf{r}_i\otimes \mathbf{Z}_i^T\mathbf{r}_i\right)$$

Hence, the *initial value of $\Sigma$* is


\begin{align}
\text{vec}\mathbf{\Sigma}^{(0)}=\left(\sum_{i=1}^n\mathbf{Z}_i^T\mathbf{Z}_i\otimes\mathbf{Z}_i^T\mathbf{Z}_i\right)^{-1}\left(\sum_{i=1}^n\mathbf{Z}_i^T\mathbf{r}_i\otimes \mathbf{Z}_i^T\mathbf{r}_i\right)
\end{align}

In [114]:
"""
    init_ls!(m::LmmModel)

Initialize parameters of a `LmmModel` object from the least squares estimate. 
`m.β`, `m.L`, and `m.σ²` are overwritten with the least squares estimates.
"""
function init_ls!(m::LmmModel{T}) where T <: AbstractFloat
    p, q = size(m.data[1].X, 2), size(m.data[1].Z, 2)
    # TODO: fill m.β, m.L, m.σ² by LS estimates
    # sleep(1e-3) # pretend this takes 1ms
    n = length(m.data)
    
    fill!(m.xty, 0.0)
    fill!(m.ztr, 0.0)
    fill!(m.ztr2, 0.0)
    fill!(m.xtx, 0.0)
    fill!(m.ztz2, 0.0)
    fill!(m.β, 0.0)
    fill!(m.L, 0.0)
    fill!(m.σ², 0.0)
    sum_n = 0
    
    for i in 1:n
        obs = m.data[i]
        
        # β
        axpy!(T(1), obs.xty, m.xty)
        axpy!(T(1), obs.xtx, m.xtx)
        m.σ²[1] += obs.yty
    end
    mul!(m.β, m.xtx \ I, m.xty)
    m.σ²[1] += dot(m.β, BLAS.gemv!('N', T(1), m.xtx, m.β, T(-2), m.xty))
    
    for i in 1:n
        obs = m.data[i]
        
        # σ²
        #copy!(m.xty, obs.xty)
        #m.σ²[1] += obs.yty + dot(m.β, BLAS.gemv!('N', T(1), obs.xtx, m.β, T(-2), m.xty))
        sum_n += size(obs.y, 1)
        
        # 𝚺
        ## Ztr \otimes Ztr
        copy!(m.ztr, obs.zty)
        BLAS.gemv!('N', T(1), obs.ztx, m.β, T(-1), m.ztr)
        axpy!(T(1), kron(m.ztr, m.ztr), m.ztr2)
        ## ZtZ \otimes ZtZ
        axpy!(T(1), kron(obs.ztz, obs.ztz), m.ztz2)
    end
    
    rdiv!(m.σ², sum_n)
    LAPACK.posv!('L', m.ztz2, m.ztr2)
    m.Σ .= reshape(m.ztr2, (q, q))
    m.L .= Matrix(cholesky(Symmetric(m.Σ)).L)
    
    m
end

init_ls!

In [115]:
init_ls!(lmm)
@show logl!(lmm)
@show lmm.β
@show lmm.σ²
@show lmm.L;

logl!(lmm) = -3.3529912271664594e6
lmm.β = [0.12814248060847544, 6.497945035096693, -3.5021387747701356, 1.0026609224782135, 5.002697971721907]
lmm.σ² = [5.70170988595954]
lmm.L = [1.442089600524507 0.0 0.0; 0.03830200684866265 1.0546394793870688 0.0; 0.027886674564416906 0.06103499518749493 1.0014753586943541]


### Correctness

Your start points should have a log-likelihood larger than -3.352991e6 (10 pts). The points you get are

In [116]:
# this is the points you get
(logl!(lmm) >  -3.352991e6) * 10

0

### Efficiency

The start point should be computed quickly. Otherwise there is no point using it as a starting point. You get full credit (10 pts) if the median run time is within 1ms.

In [117]:
bm_init = @benchmark init_ls!($lmm)

BenchmarkTools.Trial: 
  memory estimate:  1.12 MiB
  allocs estimate:  8012
  --------------
  minimum time:     370.750 μs (0.00% GC)
  median time:      388.334 μs (0.00% GC)
  mean time:        441.809 μs (11.59% GC)
  maximum time:     15.370 ms (96.71% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [118]:
# this is the points you get
clamp(1 / (median(bm_init).time / 1e6) * 10, 0, 10)

10.0

## Q6. NLP via MathProgBase.jl

We define the NLP problem using the modelling tool MathProgBase.jl. Start-up code is given below. Modify if necessary to accomodate your own code.

In [121]:
"""
    fit!(m::LmmModel, solver=Ipopt.IpoptSolver(print_level=5))

Fit an `LmmModel` object by MLE using a nonlinear programming solver. Start point 
should be provided in `m.β`, `m.σ²`, `m.L`.
"""
function fit!(
        m::LmmModel,
        solver=Ipopt.IpoptSolver(print_level=5)
    )
    p    = size(m.data[1].X, 2)
    q    = size(m.data[1].Z, 2)
    npar = p + ((q * (q + 1)) >> 1) + 1
    optm = MathProgBase.NonlinearModel(solver)
    # set lower bounds and upper bounds of parameters
    # diagonal entries of Cholesky factor L should be >= 0
    lb   = fill(-Inf, npar)
    ub   = fill( Inf, npar)
    offset = p + 1
    for j in 1:q, i in j:q
        i == j && (lb[offset] = 0)
        offset += 1
    end
    # σ² should be >= 0
    lb[end] = 0
    MathProgBase.loadproblem!(optm, npar, 0, lb, ub, Float64[], Float64[], :Max, m)
    # starting point
    par0 = zeros(npar)
    modelpar_to_optimpar!(par0, m)
    MathProgBase.setwarmstart!(optm, par0)
    # optimize
    MathProgBase.optimize!(optm)
    optstat = MathProgBase.status(optm)
    optstat == :Optimal || @warn("Optimization unsuccesful; got $optstat")
    # update parameters and refresh gradient
    optimpar_to_modelpar!(m, MathProgBase.getsolution(optm))
    logl!(m, true)
    m
end

"""
    modelpar_to_optimpar!(par, m)

Translate model parameters in `m` to optimization variables in `par`.
"""
function modelpar_to_optimpar!(
        par :: Vector,
        m   :: LmmModel
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    # β
    copyto!(par, m.β)
    # L
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        par[offset] = m.L[i, j]
        offset += 1
    end
    # σ²
    par[end] = m.σ²[1]
    par
end

"""
    optimpar_to_modelpar!(m, par)

Translate optimization variables in `par` to the model parameters in `m`.
"""
function optimpar_to_modelpar!(
        m   :: LmmModel, 
        par :: Vector
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    # β
    copyto!(m.β, 1, par, 1, p)
    # L
    fill!(m.L, 0)
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        m.L[i, j] = par[offset]
        offset   += 1
    end
    # σ²
    m.σ²[1] = par[end]    
    m
end

function MathProgBase.initialize(
        m                  :: LmmModel, 
        requested_features :: Vector{Symbol}
    )
    for feat in requested_features
        if !(feat in [:Grad])
            error("Unsupported feature $feat")
        end
    end
end

MathProgBase.features_available(m::LmmModel) = [:Grad]

function MathProgBase.eval_f(
        m   :: LmmModel, 
        par :: Vector
    )
    optimpar_to_modelpar!(m, par)
    logl!(m, false) # don't need gradient here
end

function MathProgBase.eval_grad_f(
        m    :: LmmModel, 
        grad :: Vector, 
        par  :: Vector
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    optimpar_to_modelpar!(m, par) 
    obj = logl!(m, true)
    # gradient wrt β
    copyto!(grad, m.∇β)
    # gradient wrt L
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        grad[offset] = m.∇L[i, j]
        offset += 1
    end
    # gradient with respect to σ²
    grad[end] = m.∇σ²[1]
    # return objective
    obj
end

MathProgBase.eval_g(m::LmmModel, g, par) = nothing
MathProgBase.jac_structure(m::LmmModel) = Int[], Int[]
MathProgBase.eval_jac_g(m::LmmModel, J, par) = nothing
print("done")

done

## Q7. (20 pts) Test drive

Now we can run our NLP solver to compute the MLE. For grading purpose, we first use the `:LD_LBFGS` (limited-memory BFGS) algorithm in NLopt.jl here.

In [122]:
# initialize from least squares
init_ls!(lmm)
println("objective value at starting point: ", logl!(lmm)); println()

@time fit!(lmm, NLopt.NLoptSolver(algorithm =:LD_LBFGS, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval=10000));

println("objective value at solution: ", logl!(lmm)); println()
println("solution values:")
@show lmm.β
@show lmm.σ²
@show lmm.L * transpose(lmm.L)
println("gradient @ solution:")
@show lmm.∇β
@show lmm.∇σ²
@show lmm.∇L
@show sqrt(abs2(norm(lmm.∇β)) + abs2(norm(lmm.∇σ²) + 
        abs2(norm(LowerTriangular(lmm.∇L)))))

objective value at starting point: -3.3529912271664594e6

  0.377191 seconds (465.10 k allocations: 25.187 MiB, 78.13% compilation time)
objective value at solution: -2.8342645025174716e6

solution values:
lmm.β = [0.12444023996787072, 6.4983368726252575, -3.5005134232462125, 1.0018216612192339, 5.001953229483336]
lmm.σ² = [1.5023519585474876]
lmm.L * transpose(lmm.L) = [2.06840455370014 0.05309337211277105 0.032613129503242945; 0.05309337211277105 1.121372485988842 0.05588413884451682; 0.032613129503242945 0.05588413884451682 1.0123726731322937]
gradient @ solution:
lmm.∇β = [-0.2873181251507262, 0.4782329138438399, 0.42684352436288364, 0.34273359166486017, 0.5610356300444153]
lmm.∇σ² = [0.04222059449278959]
lmm.∇L = [0.05363972654592242 0.002817757375522449 0.04870699557127027; 0.014761330574561499 0.35590513452657424 0.09362627075883101; 0.06578072471073763 0.08168002342096178 -0.3274518393272249]
sqrt(abs2(norm(lmm.∇β)) + abs2(norm(lmm.∇σ²) + abs2(norm(LowerTriangular(lmm.∇L))))) =

1.0049385907425816

### Correctness

You get 10 points if the following code does not throw `AssertError`.

In [123]:
# objective at solution should be close enough to the optimal
@assert logl!(lmm) > -2.83427e6
# gradient at solution should be small enough
@assert sqrt(abs2(norm(lmm.∇β)) + abs2(norm(lmm.∇σ²) + 
        abs2(norm(LowerTriangular(lmm.∇L))))) < 0.1

LoadError: AssertionError: sqrt(abs2(norm(lmm.∇β)) + abs2(norm(lmm.∇σ²) + abs2(norm(LowerTriangular(lmm.∇L))))) < 0.1

### Efficiency

My median run time is 132.5ms. You get 10 points if your median time is within 1s(=1000ms).

In [77]:
bm_bfgs = @benchmark fit!($lmm, $(NLopt.NLoptSolver(algorithm=:LD_LBFGS, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval = 10000))) setup = (init_ls!(lmm))

BenchmarkTools.Trial: 
  memory estimate:  10.17 KiB
  allocs estimate:  209
  --------------
  minimum time:     82.953 ms (0.00% GC)
  median time:      85.040 ms (0.00% GC)
  mean time:        85.256 ms (0.00% GC)
  maximum time:     88.638 ms (0.00% GC)
  --------------
  samples:          59
  evals/sample:     1

In [78]:
# this is the points you get
clamp(1 / (median(bm_bfgs).time / 1e9) * 10, 0, 10)

10.0

## Q8. (10 pts) Gradient free vs gradient-based methods

Advantage of using a modelling tool such as MathProgBase.jl is that we can easily switch the backend solvers. For a research problem, we never know beforehand which solver works best. 

Try different solvers in the NLopt.jl and Ipopt.jl packages. Compare the results in terms run times (the shorter the better), objective values at solution (the larger the better), and gradients at solution (closer to 0 the better). Summarize what you find.

See this [page](https://nlopt.readthedocs.io/en/latest/NLopt_Algorithms/) for the descriptions of algorithms in NLopt.

Documentation for the Ipopt can be found [here](https://coin-or.github.io/Ipopt/). 

In [79]:
# vector of solvers to compare
solvers = [
    # NLopt: gradient-based algorithms
    NLopt.NLoptSolver(algorithm=:LD_LBFGS, 
        ftol_rel = 1e-12, ftol_abs = 1e-12,
        xtol_rel = 1e-12, xtol_abs = 1e-12,
        maxeval=10000),
    NLopt.NLoptSolver(algorithm=:LD_MMA, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval=10000),
    # NLopt: gradient-free algorithms
    NLopt.NLoptSolver(algorithm=:LN_BOBYQA, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval=10000),
    # Ipopt
    Ipopt.IpoptSolver(print_level=0)
]
# containers for results
runtime = zeros(length(solvers))
objvals = zeros(length(solvers))
gradnrm = zeros(length(solvers))

for (i, solver) in enumerate(solvers)
    bm = @benchmark fit!($lmm, $solver) setup = (init_ls!(lmm))
    runtime[i] = median(bm).time / 1e9
    objvals[i] = logl!(lmm, true)
    gradnrm[i] = sqrt(abs2(norm(lmm.∇β)) + abs2(norm(lmm.∇σ²) + 
        abs2(norm(LowerTriangular(lmm.∇L)))))
end


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



In [80]:
DataFrame(Runtime = runtime, Objective = objvals, Gradnorm = gradnrm)

,Runtime,Objective,Gradnorm
,Float64,Float64,Float64
1,0.081528,-2.83426e6,1.00494
2,0.118904,-2.83426e6,0.0052726
3,0.0434311,-2.83539e6,31729.5
4,3.72033,-2.83426e6,2.43135e-6


## Q9. (10 pts) Compare with existing art

Let's compare our method with lme4 package in R and MixedModels.jl package in Julia. Both lme4 and MixedModels.jl are developed mainly by Doug Bates. Summarize what you find.

In [81]:
method  = ["257", "lme4", "MixedModels.jl"]
runtime = zeros(3)  # record the run times
loglike = zeros(3); # record the log-likelihood at MLE

### Your approach

In [82]:
bm_257 = @benchmark fit!($lmm, $(NLopt.NLoptSolver(algorithm=:LD_MMA, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval=10000))) setup=(init_ls!(lmm))
runtime[1] = (median(bm_257).time) / 1e9
loglike[1] = logl!(lmm)

-2.834264502360534e6

### lme4 

In [86]:
R"""
library(lme4)
library(readr)
library(magrittr)

testdata <- read_csv("lmm_data.txt")
"""

RObject{StrSxp}
 [1] "magrittr"  "readr"     "lme4"      "Matrix"    "stats"     "graphics" 
 [7] "grDevices" "utils"     "datasets"  "methods"   "base"     


In [ ]:
R"""
rtime <- system.time(mmod <- 
  lmer(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID), testdata, REML = FALSE))
"""

In [ ]:
R"""
rtime <- rtime["elapsed"]
summary(mmod)
rlogl <- logLik(mmod)
"""
runtime[2] = @rget rtime
loglike[2] = @rget rlogl;

### MixedModels.jl



In [87]:
testdata = CSV.File("lmm_data.txt", types = Dict(1=>String)) |> DataFrame!

,ID,Y,X1,X2,X3,X4,Z1,Z2
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,2.26056,-0.358432,0.379066,-0.106696,0.783914,-1.27916,-1.9616
2,1,10.4825,-0.592442,-0.540181,1.13762,1.46277,-0.746512,1.26418
3,1,-13.5503,-1.91127,0.71865,0.757802,-0.86518,-1.69606,0.088986
4,1,5.29505,-0.106051,-0.16981,0.279181,0.741984,-0.0258785,0.827073
5,1,11.0453,-0.0209066,-1.7958,1.72241,0.760698,-0.561809,-1.46954
6,1,-0.31277,0.56542,1.98757,-0.437112,-0.102006,1.26906,1.382
7,1,-0.688038,-0.451169,0.905654,2.23043,-0.386278,-1.83272,0.276094
8,1,-4.50433,-1.37714,-0.527188,-0.0893463,0.210053,1.19325,1.49268
9,1,8.48327,1.19613,0.147975,1.18613,0.171897,2.21632,1.10782


In [88]:
mj = fit(MixedModel, @formula(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID)), testdata)
bm_mm = @benchmark fit(MixedModel, @formula(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID)), testdata)
loglike[3] = loglikelihood(mj)
runtime[3] = median(bm_mm).time / 1e9

2.9646297295

In [89]:
display(bm_mm)
mj

BenchmarkTools.Trial: 
  memory estimate:  879.30 MiB
  allocs estimate:  5748
  --------------
  minimum time:     2.954 s (0.40% GC)
  median time:      2.965 s (1.10% GC)
  mean time:        2.965 s (1.10% GC)
  maximum time:     2.975 s (1.81% GC)
  --------------
  samples:          2
  evals/sample:     1

|             |    Est. |     SE |        z |      p |   σ_ID |
|:----------- | -------:| ------:| --------:| ------:| ------:|
| (Intercept) |  0.1238 | 0.0455 |     2.72 | 0.0064 | 1.4383 |
| X1          |  6.4983 | 0.0009 |  6989.59 | <1e-99 |        |
| X2          | -3.5005 | 0.0009 | -3765.89 | <1e-99 |        |
| X3          |  1.0018 | 0.0009 |  1077.25 | <1e-99 |        |
| X4          |  5.0020 | 0.0009 |  5377.18 | <1e-99 |        |
| Z2          |         |        |          |        | 1.0061 |
| Z1          |         |        |          |        | 1.0591 |
| Residual    |  1.2257 |        |          |        |        |


### Summary

In [90]:
DataFrame(method = method, runtime = runtime, logl = loglike)

,method,runtime,logl
,String,Float64,Float64
1,257,0.117249,-2.83426e6
2,lme4,0.0,0.0
3,MixedModels.jl,2.96463,-2.83426e6


## Q10. Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on complex longitudinal data sets with millions of records. And you beat current software by XXX fold. 